## Code to generate an array of parameter inputs

For the identified high, medium, and low values of the input parameters, this notebook generates the complete set of combinations and batches them. The sets and batches are then written in that organized order to csv files. These files serve as inputs to the loop simulation, allowing the simualtions to be run in parallel.

This notebook also serves to identify simulations that were not run and then regroups the simulations to isolate those that haven't been run yet. This allowed all simulations to be ran.

Last updated May 8, 2024 by Madeline Murphy

In [22]:
import Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("PyPlot")
Pkg.add("NBInclude")
Pkg.add("Trapz")
Pkg.add("Printf")
Pkg.add("Statistics")
Pkg.add("NBInclude")
Pkg.add("DifferentialEquations")
Pkg.add("Random")

   Resolving package versions...
  No Changes to `/panfs/jay/groups/16/dauenha0/murp1677/.julia/environments/v1.9/Project.toml`
  No Changes to `/panfs/jay/groups/16/dauenha0/murp1677/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `/panfs/jay/groups/16/dauenha0/murp1677/.julia/environments/v1.9/Project.toml`
  No Changes to `/panfs/jay/groups/16/dauenha0/murp1677/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `/panfs/jay/groups/16/dauenha0/murp1677/.julia/environments/v1.9/Project.toml`
  No Changes to `/panfs/jay/groups/16/dauenha0/murp1677/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `/panfs/jay/groups/16/dauenha0/murp1677/.julia/environments/v1.9/Project.toml`
  No Changes to `/panfs/jay/groups/16/dauenha0/murp1677/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `/panfs/jay/groups/16/dauenha0/murp1677/.julia/environm

In [1]:
# Load packages
using CSV, DataFrames, Trapz, Printf, NBInclude, DifferentialEquations, Statistics, Random

In [2]:
function AllCombinations(v...)
    allcombo = vec(collect(Iterators.product(v...)))
    return allcombo
end

AllCombinations (generic function with 1 method)

In [3]:
function RxnParameters(parameters, BEa)
 # Constants
        T = 298.15             # temperature [K]
        kB = 8.61733034e-5        # Boltzmann constant [eV/K]
        h = 4.1357e-15            # Planck constant [eV-s]
        ev = 1.602e-19            # Converts Joules to Ev
         
        # Unpacking Parameters
        gamma_ab = parameters[7]
        gamma_ac = parameters[8]
        
        delta_ab = parameters[9]
        delta_ac = parameters[10]
        
        alpha_a = parameters[1]
        alpha_b = parameters[2]
        alpha_c = parameters[3]
        
        beta_a = parameters[4]
        beta_b = parameters[5]
        beta_c = parameters[6]
    
        delBEa = parameters[11]
        
    
        ## Binding Energies from linear scaling relationships
        BEb = gamma_ab*BEa + (1-gamma_ab)*delta_ab
        BEc = gamma_ac*BEa + (1-gamma_ac)*delta_ac
    
        ## Surface Reactions 
        deltaG1 = Vector{Float64}(undef,3)
        deltaG1[1] = - BEb - (-BEa)
        deltaG1[2] = - BEc - (-BEb)
        deltaG1[3] = - BEa - (-BEc)
        
        ## Activation Energies from BEP relationships
        Ea1 = Vector{Float64}(undef,3)
        Ea1[1] = alpha_a*deltaG1[1] + beta_a
        Ea1[2] = alpha_b*deltaG1[2] + beta_b
        Ea1[3] = alpha_c*deltaG1[3] + beta_c 
  
        # Forward rate constant via transition state theory (TST)
        kf1 = (kB*T/h)*exp.(-Ea1/(kB*T))
        
        # Equilibrium constant
        K1 = exp.(-deltaG1/(kB*T))
        
        # Reverse rate constant
        kr1 = kf1./K1
    
     ## STATE 2
    BEa = BEa+delBEa
    
        ## Binding Energies from linear scaling relationships
        BEb = gamma_ab*BEa + (1-gamma_ab)*delta_ab
        BEc = gamma_ac*BEa + (1-gamma_ac)*delta_ac
    
        ## Surface Reactions 
        deltaG2 = Vector{Float64}(undef,3)
        deltaG2[1] = - BEb - (-BEa)
        deltaG2[2] = - BEc - (-BEb)
        deltaG2[3] = - BEa - (-BEc)
        
        ## Activation Energies from BEP relationships
        Ea2 = Vector{Float64}(undef,3)
        Ea2[1] = alpha_a*deltaG2[1] + beta_a
        Ea2[2] = alpha_b*deltaG2[2] + beta_b
        Ea2[3] = alpha_c*deltaG2[3] + beta_c 
  
        # Forward rate constant via transition state theory (TST)
        kf2 = (kB*T/h)*exp.(-Ea2/(kB*T))
        
        # Equilibrium constant
        K2 = exp.(-deltaG2/(kB*T))
        
        # Reverse rate constant
        kr2 = kf2./K2
    
        Ea = vcat(Ea1,Ea2)
    return Ea
end 

RxnParameters (generic function with 1 method)

In [4]:
alpha = [0.2,0.6,0.9]
beta = [0.6,0.9,1.2]
gamma = [0.6,1.4,1.8]
delta = [0.5,1.0,1.5]
delBEa = [0.3,0.5,0.8]
BEa = 0.8


allcombo = AllCombinations(alpha,alpha,alpha,beta,beta,beta,gamma,gamma,delta,delta,delBEa)

177147-element Vector{NTuple{11, Float64}}:
 (0.2, 0.2, 0.2, 0.6, 0.6, 0.6, 0.6, 0.6, 0.5, 0.5, 0.3)
 (0.6, 0.2, 0.2, 0.6, 0.6, 0.6, 0.6, 0.6, 0.5, 0.5, 0.3)
 (0.9, 0.2, 0.2, 0.6, 0.6, 0.6, 0.6, 0.6, 0.5, 0.5, 0.3)
 (0.2, 0.6, 0.2, 0.6, 0.6, 0.6, 0.6, 0.6, 0.5, 0.5, 0.3)
 (0.6, 0.6, 0.2, 0.6, 0.6, 0.6, 0.6, 0.6, 0.5, 0.5, 0.3)
 (0.9, 0.6, 0.2, 0.6, 0.6, 0.6, 0.6, 0.6, 0.5, 0.5, 0.3)
 (0.2, 0.9, 0.2, 0.6, 0.6, 0.6, 0.6, 0.6, 0.5, 0.5, 0.3)
 (0.6, 0.9, 0.2, 0.6, 0.6, 0.6, 0.6, 0.6, 0.5, 0.5, 0.3)
 (0.9, 0.9, 0.2, 0.6, 0.6, 0.6, 0.6, 0.6, 0.5, 0.5, 0.3)
 (0.2, 0.2, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.5, 0.5, 0.3)
 (0.6, 0.2, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.5, 0.5, 0.3)
 (0.9, 0.2, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.5, 0.5, 0.3)
 (0.2, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.5, 0.5, 0.3)
 ⋮
 (0.2, 0.9, 0.6, 1.2, 1.2, 1.2, 1.8, 1.8, 1.5, 1.5, 0.8)
 (0.6, 0.9, 0.6, 1.2, 1.2, 1.2, 1.8, 1.8, 1.5, 1.5, 0.8)
 (0.9, 0.9, 0.6, 1.2, 1.2, 1.2, 1.8, 1.8, 1.5, 1.5, 0.8)
 (0.2, 0.2, 0.9, 1.2, 1.2, 1.2, 1.8, 1.8,

In [5]:
count = 0
delete_idx = Vector{Int}(undef,177147)
for i in range(1,length(allcombo))
    Ea = RxnParameters(allcombo[i],BEa);
    if any(Ea.<0.0)
        delete_idx[count+1] = i
        count = count +1
    end
end
delete_idx = delete_idx[1:count]
deleteat!(allcombo, delete_idx)
println(count)
println(length(allcombo))

2835
174312


In [6]:
# define sets
set1 = allcombo[1:90000]

set2 = allcombo[90001:end]

84312-element Vector{NTuple{11, Float64}}:
 (0.2, 0.2, 0.6, 0.6, 0.9, 0.9, 1.4, 1.8, 1.0, 1.0, 0.5)
 (0.6, 0.2, 0.6, 0.6, 0.9, 0.9, 1.4, 1.8, 1.0, 1.0, 0.5)
 (0.9, 0.2, 0.6, 0.6, 0.9, 0.9, 1.4, 1.8, 1.0, 1.0, 0.5)
 (0.2, 0.6, 0.6, 0.6, 0.9, 0.9, 1.4, 1.8, 1.0, 1.0, 0.5)
 (0.6, 0.6, 0.6, 0.6, 0.9, 0.9, 1.4, 1.8, 1.0, 1.0, 0.5)
 (0.9, 0.6, 0.6, 0.6, 0.9, 0.9, 1.4, 1.8, 1.0, 1.0, 0.5)
 (0.2, 0.9, 0.6, 0.6, 0.9, 0.9, 1.4, 1.8, 1.0, 1.0, 0.5)
 (0.6, 0.9, 0.6, 0.6, 0.9, 0.9, 1.4, 1.8, 1.0, 1.0, 0.5)
 (0.9, 0.9, 0.6, 0.6, 0.9, 0.9, 1.4, 1.8, 1.0, 1.0, 0.5)
 (0.2, 0.2, 0.9, 0.6, 0.9, 0.9, 1.4, 1.8, 1.0, 1.0, 0.5)
 (0.6, 0.2, 0.9, 0.6, 0.9, 0.9, 1.4, 1.8, 1.0, 1.0, 0.5)
 (0.9, 0.2, 0.9, 0.6, 0.9, 0.9, 1.4, 1.8, 1.0, 1.0, 0.5)
 (0.2, 0.6, 0.9, 0.6, 0.9, 0.9, 1.4, 1.8, 1.0, 1.0, 0.5)
 ⋮
 (0.2, 0.9, 0.6, 1.2, 1.2, 1.2, 1.8, 1.8, 1.5, 1.5, 0.8)
 (0.6, 0.9, 0.6, 1.2, 1.2, 1.2, 1.8, 1.8, 1.5, 1.5, 0.8)
 (0.9, 0.9, 0.6, 1.2, 1.2, 1.2, 1.8, 1.8, 1.5, 1.5, 0.8)
 (0.2, 0.2, 0.9, 1.2, 1.2, 1.2, 1.8, 1.8, 

In [7]:
function identify_reruns_SimID(allcombo, csv_path)
    # Read the CSV file
    len = length(allcombo)
    data = CSV.File(csv_path) |> DataFrame

    # Get unique Simulation IDs
    sim_ids = data."Simulation ID"
    println(length(sim_ids))

    Rerun_ID = Int[]
    rerun_params = []

    for j in range(1,1500) # batch number
        for k in range(1,60) # job ID 
            
            ID = (j-1)*60 + k
            
            if ID > len
                break
            end
            
            combo = allcombo[ID]

            if !(ID in sim_ids)
                push!(Rerun_ID, ID)
                push!(rerun_params, (j, k, combo...))
            end
        end
    end

    return rerun_params
end


identify_reruns_SimID (generic function with 1 method)

In [8]:
function identify_reruns_SimID_2(allcombo, csv_path)
    # Read the CSV file
    len = length(allcombo)
    data = CSV.File(csv_path) |> DataFrame

    # Get unique Simulation IDs
    sim_ids = data."Simulation ID"
    println(length(sim_ids))

    Rerun_ID = Int[]
    rerun_params = []

    for j in range(1,1406) # batch number
        for k in range(1,60) # job ID 
            
            index = (j-1)*60 + k 
            ID = (j-1)*60 + k + 90000
            
            if ID > len + 90000
                break
            end
            
            combo = allcombo[index]

            if !(ID in sim_ids)
                push!(Rerun_ID, ID)
                push!(rerun_params, (j, k, combo...))
            end
        end
    end

    return rerun_params
end


identify_reruns_SimID_2 (generic function with 1 method)

In [13]:
# Identify Reruns 
csv_path = "/home/dauenha0/murp1677/Cyclic_Dynamics/Results/2024-03-13_ALL_Simulation_outputs.csv"

Rerun_params_set1 = identify_reruns_SimID(set1, csv_path)
Rerun_params_set1 = shuffle(Rerun_params_set1)
println(length(Rerun_params_set1))
println((Rerun_params_set1[1:10,:]))

174261
51
Any[(1427, 28, 0.2, 0.9, 0.9, 0.9, 0.6, 0.9, 1.4, 0.6, 1.0, 1.0, 0.5); (1476, 26, 0.6, 0.2, 0.9, 1.2, 0.6, 0.9, 1.8, 1.4, 1.0, 1.0, 0.5); (1450, 15, 0.9, 0.6, 0.6, 0.9, 1.2, 0.6, 0.6, 1.4, 1.0, 1.0, 0.5); (1452, 15, 0.9, 0.9, 0.9, 1.2, 0.6, 0.9, 0.6, 1.4, 1.0, 1.0, 0.5); (1451, 25, 0.2, 0.6, 0.2, 0.9, 0.6, 0.9, 0.6, 1.4, 1.0, 1.0, 0.5); (1452, 20, 0.6, 0.6, 0.2, 0.6, 0.9, 0.9, 0.6, 1.4, 1.0, 1.0, 0.5); (1480, 55, 0.2, 0.2, 0.9, 0.6, 0.9, 1.2, 1.8, 1.4, 1.0, 1.0, 0.5); (1443, 16, 0.2, 0.2, 0.2, 0.9, 0.6, 1.2, 1.8, 0.6, 1.0, 1.0, 0.5); (1476, 60, 0.9, 0.9, 0.9, 0.6, 0.9, 0.9, 1.8, 1.4, 1.0, 1.0, 0.5); (1432, 26, 0.6, 0.9, 0.9, 0.6, 0.9, 1.2, 1.4, 0.6, 1.0, 1.0, 0.5);;]


In [14]:
## SET 1
#initialize count
count = 1

    for j in range(1,18) # batch number
        for k in range(1,111) # job ID 
            combo = Rerun_params_set1[count]

            # Create DataFrame to export parameters to CSV
            fpath = "/home/dauenha0/murp1677/Cyclic_Dynamics/Batch_Scripts/Parameters/ReRuns_Round19_Set1/ReRun_Batch$(j).csv"
            df = DataFrame(batch_number=combo[1], jobID=combo[2], alpha_a = combo[3], alpha_b = combo[4], 
            alpha_c = combo[5], beta_a = combo[6], beta_b = combo[7], beta_c = combo[8], gamma_BA = combo[9], 
            gamma_CA = combo[10], delta_BA = combo[11], delta_CA = combo[12], delBEa = combo[13])

            if isfile(fpath) == false # checks if file exists
            # if file does NOT exist, write file and include column names
                col_names = ["BatchID", "JobID", "alpha_a", "alpha_b", "alpha_c", "beta_a", "beta_b", "beta_c", "gamma_BA", "gamma_CA", "delta_BA", "delta_CA", "delBEa"]; # 13 columns
                CSV.write(fpath,df, header=col_names)
            else 
                CSV.write(fpath,df, append=true)
            end
            count = count + 1
            if count > length(Rerun_params_set1)
                println("Set 1 Exported")
                break
            end
        end
    end
println(count-1)


Set 1 Exported


LoadError: BoundsError: attempt to access 51-element Vector{Any} at index [52]

In [33]:
csv_path = "/home/dauenha0/murp1677/Cyclic_Dynamics/Results/2024-03-12_ALL_Simulation_outputs.csv"

# Read the CSV file
len = length(allcombo)
println(len)

data = CSV.File(csv_path) |> DataFrame
# Get unique Simulation IDs
sim_ids = data."Simulation ID"
println(length(sim_ids))
Rerun_ID = []

for i in range(1,len)
    if !(i in sim_ids)
        push!(Rerun_ID, i)
    end
end

#println(Rerun_ID)

174312
171734
Any[84380, 84382, 84385, 84387, 84388, 84389, 84390, 84393, 84395, 84396, 84398, 84399, 84401, 84403, 84404, 84405, 84406, 84408, 84409, 84410, 84411, 84418, 84419, 84420, 84427, 84428, 84429, 84431, 84432, 84433, 84434, 84436, 84437, 84438, 84439, 84441, 84444, 84447, 84448, 84449, 84450, 84453, 84457, 84459, 84460, 84464, 84465, 84466, 84467, 84469, 84470, 84471, 84472, 84473, 84474, 84475, 84476, 84477, 84478, 84481, 84482, 84484, 84485, 84486, 84487, 84489, 84490, 84491, 84492, 84493, 84494, 84495, 84496, 84498, 84499, 84501, 84502, 84503, 84504, 84505, 84506, 84512, 84513, 84514, 84515, 84516, 84517, 84518, 84519, 84520, 84521, 84522, 84523, 84524, 84525, 84526, 84527, 84528, 84529, 84530, 84531, 84532, 84533, 84534, 84535, 84536, 84537, 84538, 84539, 84540, 84542, 84543, 84544, 84545, 84546, 84547, 84548, 84549, 84550, 84551, 84552, 84673, 84674, 84675, 84676, 84677, 84678, 84679, 84680, 84681, 84682, 84683, 84684, 84685, 84686, 84687, 84688, 84689, 84690, 84691, 84

In [28]:
# Identify Reruns 
csv_path = "/home/dauenha0/murp1677/Cyclic_Dynamics/Results/2024-03-12_ALL_Simulation_outputs.csv"

Rerun_params_set2 = identify_reruns_SimID_2(set2, csv_path)
Rerun_params_set2 = shuffle(Rerun_params_set2)
println(length(Rerun_params_set2))
println((Rerun_params_set2[1:10,:]))

171734
0


LoadError: BoundsError: attempt to access 0-element Vector{Any} at index [1:10, 1:1]

In [25]:
## SET 2
#initialize count
count = 1

    for j in range(1,18) # batch number
        for k in range(1,111) # job ID 
            combo = Rerun_params_set2[count]

            # Create DataFrame to export parameters to CSV
            fpath = "/home/dauenha0/murp1677/Cyclic_Dynamics/Batch_Scripts/Parameters/ReRuns_Round15_Set2/ReRun_Batch$(j).csv"
            df = DataFrame(batch_number=combo[1], jobID=combo[2], alpha_a = combo[3], alpha_b = combo[4], 
            alpha_c = combo[5], beta_a = combo[6], beta_b = combo[7], beta_c = combo[8], gamma_BA = combo[9], 
            gamma_CA = combo[10], delta_BA = combo[11], delta_CA = combo[12], delBEa = combo[13])

            if isfile(fpath) == false # checks if file exists
            # if file does NOT exist, write file and include column names
                col_names = ["BatchID", "JobID", "alpha_a", "alpha_b", "alpha_c", "beta_a", "beta_b", "beta_c", "gamma_BA", "gamma_CA", "delta_BA", "delta_CA", "delBEa"]; # 13 columns
                CSV.write(fpath,df, header=col_names)
            else 
                CSV.write(fpath,df, append=true)
            end
            count = count + 1
            if count >= length(Rerun_params_set2)
                println("Set 2 Exported")
                break
            end
        end
    end
print(count)

Set 2 Exported
Set 2 Exported


LoadError: BoundsError: attempt to access 22-element Vector{Any} at index [23]

In [16]:
# Function to create a folder in Jupyter Notebooks using Julia
function create_folder(folder_name::AbstractString, path::AbstractString="")
    full_path = joinpath(path, folder_name)
    try
        mkdir(full_path)
        println("Folder '$folder_name' created at: $full_path")
    catch
        println("Error: Failed to create folder '$folder_name'")
    end
end


create_folder (generic function with 2 methods)